# 1.Setting up the environment
This involves installing all the required packages and limiting the GPU usage growth

In [1]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
alabaster                     0.7.12
anaconda-client               1.11.0
anaconda-navigator            2.3.2
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.11.7
astropy                       5.1
astunparse                    1.6.3
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.11.1
binaryornot                   0.4.4
bi

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4233196721562717544
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2258055988
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9939576048862961524
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
# importing tensorflow package
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [4]:
# importing other packages
import numpy as np
import os
from matplotlib import pyplot as plt
import pandas as pd
import math
import random
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LeakyReLU,Dropout,Input,ReLU,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError, BinaryCrossentropy
from tensorflow.keras.metrics import BinaryCrossentropy as bce, RootMeanSquaredError as rmse
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback
from openpyxl import load_workbook

# 2. Importing the dataset

The dataset contains the positions of centres of the 10 magnetostrictive spheres of radius 0.4 cm dispersed throughout the piezoelectric matrix and the corresponding ME coupling coefficients.

In [ ]:
train_data = pd.read_csv("ME Composite Dataset Actual 2.csv")
train_data_filter = train_data[:][train_data["Reject"] == False]
train_data_filter = train_data_filter.reset_index(drop=True)

In [ ]:
train_data_filter.shape

# 3.Building ANN for evaluation of design

### 3.1. ANN architecture

In [ ]:
def ANN():
    
    input_layer = Input(shape=(30,))
    
    dense1 = Dense(35)(input_layer)
    batchnorm1 = BatchNormalization()(dense1)
    leakyRelu1 = LeakyReLU(0.2)(batchnorm1)
    dropout1 = Dropout(0.1)(leakyRelu1)
    
    dense2 = Dense(40)(dropout1)
    batchnorm2 = BatchNormalization()(dense2)
    leakyRelu2 = LeakyReLU(0.2)(batchnorm2)
    dropout2 = Dropout(0.1)(leakyRelu2)
    
    
    dense3 = Dense(35)(dropout2)
    batchnorm3 = BatchNormalization()(dense3)
    leakyRelu3 = LeakyReLU(0.2)(batchnorm3)
    dropout3 = Dropout(0.1)(leakyRelu3)
    
    output_layer = Dense(1)(dropout3)
    
    model = Model(inputs=input_layer,outputs=output_layer)
    
    return model

### 3.2. Creating instance of ANN model

In [ ]:
ann = ANN()

In [ ]:
ann.summary()

### 3.3. Loading weights onto ANN(shouldn't be done for step after last design round)

In [ ]:
ann.load_weights('ann.h5')

### 3.4. Defining optimizers and losses

In [ ]:
ann_opt = Adam(learning_rate = 0.01)
ann_loss = MeanSquaredError()

### 3.5.Compiling the ANN model

In [ ]:
ann.compile(ann_opt,ann_loss,metrics=['RootMeanSquaredError'])

# 4. Building GAN

### 4.1. Building the Generator Network

In [5]:
def Generator():
    
    input_layer = Input(shape=(2,))
    
    dense1 = Dense(4)(input_layer)
    batchnorm1 = BatchNormalization()(dense1)
    leakyRelu1 = LeakyReLU(0.1)(batchnorm1)
    
    dense2 = Dense(8)(leakyRelu1)
    batchnorm2 = BatchNormalization()(dense2)
    leakyRelu2 = LeakyReLU(0.1)(batchnorm2)
    
    dense3 = Dense(16)(leakyRelu2)
    batchnorm3 = BatchNormalization()(dense3)
    leakyRelu3 = LeakyReLU(0.1)(batchnorm3)
    
    output_layer = Dense(30)(leakyRelu3)

    model = Model(inputs=input_layer,outputs=output_layer)

    return model  

In [6]:
generator = Generator()

In [7]:
generator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 4)                 12        
                                                                 
 batch_normalization (BatchN  (None, 4)                16        
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 4)                 0         
                                                                 
 dense_1 (Dense)             (None, 8)                 40        
                                                                 
 batch_normalization_1 (Batc  (None, 8)                32        
 hNormalization)                                             

### 4.2 Building the Discriminator Network

In [8]:
def Discriminator():
  
    input_layer = Input(shape=(30,))
    
    # Real or fake classification
    dense1 = Dense(16)(input_layer)
    batchnorm1 = BatchNormalization()(dense1)
    leakyRelu1 = LeakyReLU(0.1)(batchnorm1)
    
    dense2 = Dense(8)(leakyRelu1)
    batchnorm2 = BatchNormalization()(dense2)
    leakyRelu2 = LeakyReLU(0.1)(batchnorm2)    
    
    dense3 = Dense(4)(leakyRelu2)
    batchnorm3 = BatchNormalization()(dense3)
    leakyRelu3 = LeakyReLU(0.1)(batchnorm3)
    
    dense4 = Dense(2)(leakyRelu3)
    batchnorm4 = BatchNormalization()(dense4)
    leakyRelu4 = LeakyReLU(0.1)(batchnorm4)
    
    # Pass to Output layer
    output_layer = Dense(1, activation='sigmoid')(leakyRelu4)

    model = Model(inputs = input_layer, outputs=output_layer)

    return model

In [9]:
discriminator = Discriminator()

In [10]:
discriminator.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 30)]              0         
                                                                 
 dense_4 (Dense)             (None, 16)                496       
                                                                 
 batch_normalization_3 (Batc  (None, 16)               64        
 hNormalization)                                                 
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 8)                 136       
                                                                 
 batch_normalization_4 (Batc  (None, 8)                32        
 hNormalization)                                           

### 4.3. Loading weights onto generator and discriminator

In [11]:
generator.load_weights('generator.h5')
discriminator.load_weights('discriminator.h5')

### 4.4. Defining optimizers

In [12]:
g_opt = Adam(learning_rate=0.001)
d_opt = Adam(learning_rate=0.0001)
g_loss = BinaryCrossentropy()
d_loss = BinaryCrossentropy()

### 4.5. Building sub-classed model

In [13]:
class CompositeGAN(Model):

    def __init__(self,generator,discriminator, *args, **kwargs):
        
         # Pass through args and kwargs to base class 
        super().__init__(*args, **kwargs)
        
        # Creating attributes for generator and discriminator
        self.generator = generator
        self.discriminator = discriminator
  
    def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs):
        # Compile with base class
        super().compile(*args, **kwargs)
        # Creating attributes for losses and optimizers
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss
  
    def train_step(self, batch):
        # Getting the data
        real_data = batch
        fake_data = self.generator(tf.random.normal(shape=[120,2]),training=False)
        fake_data /= tf.norm(fake_data,axis=1)[:,None]
        fake_data *= 30

        # Training the discriminator
        with tf.GradientTape()  as d_tape:
            # Passing real and fake images to the discriminator model
            yhat_real = self.discriminator(real_data, training=True)
            yhat_fake = self.discriminator(fake_data, training=True)
            yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)

            # Creating labels for real and fake images
            y_realfake = tf.concat([tf.ones_like(yhat_real), tf.zeros_like(yhat_fake)], axis=0) 

            # Adding some noise to the TRUE outputs
            noise_real = -0.1*tf.random.uniform(tf.shape(yhat_real))
            noise_fake = 0.1*tf.random.uniform(tf.shape(yhat_fake))
            y_realfake += tf.concat([noise_real, noise_fake], axis = 0)

            # Calculating the loss
            total_d_loss = self.d_loss(y_realfake, yhat_realfake)
            
        # Applying backpropagation - nn learn 
        dgrad = d_tape.gradient([total_d_loss], self.discriminator.trainable_variables)
        self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))

        # Training the generator
        with tf.GradientTape() as g_tape:
            # Generating new images
            gen_data = self.generator(tf.random.normal(shape=[120,2]), training=True)
            gen_data /= tf.norm(gen_data,axis=1)[:,None]
            gen_data *= 30
            # Creating the predicted labels
            yhat_gen = self.discriminator(gen_data, training=False)
            # Calculate loss
            total_g_loss = self.g_loss(tf.ones_like(yhat_gen), yhat_gen)

        # Applying backprop
        ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))

        return {"d_loss":total_d_loss,"g_loss":total_g_loss}

In [14]:
# Create instance of subclassed model
compositegan = CompositeGAN(generator,discriminator)

In [15]:
# Compile the model
compositegan.compile(g_opt, d_opt, g_loss, d_loss)

# 5. Adaptive Algorithm

Mo = 349 <br>
No = 50 <br>
M = 120 <br>
N = 30 <br>
p = 8 <br>
Me = 360 <br>
No mutation <br>
Sequence of steps: Design round 0 - 5.2, 5.3, 5.4, 5.6, 5.7, 5.8, 5.9, 5.10, 5.11 <br>
Design rounds 1 till last: 5.12,5.3, 5.5, 5.7, 5.8, 5.9, 5.11, 5.12, 5.3 <br>
After last design round: 5.13, 5.14

### 5.1. Initializing the adaptive algorithm parameters

In [16]:
Mo = 349
No = 50
M = 120
N = 30
p = 8
Me = 360

### 5.2. Sampling data for training ANN

In [ ]:
train_data_ANN = train_data_filter.sample(No)
train_data_ANN.to_csv(r"Training data 2\ME_Composite_Design_ANN_round0.csv",index=False)
X = train_data_ANN.iloc[:,1:-2]
y = train_data_ANN.iloc[:,-2]

### 5.3. Training ANN

In [ ]:
ann_hist = ann.fit(X,y,epochs=200,batch_size = 2)

#### 5.3.1. ANN loss plot

In [ ]:
plt.suptitle('Regression loss')
plt.plot(ann_hist.history['root_mean_squared_error'], label='Regression_loss')
plt.legend()
plt.show()

### 5.4. Predicting ME coupling coefficient for the total population

In [ ]:
y_pred = pd.DataFrame(ann.predict(train_data_filter.iloc[:,1:-2]),columns = ['ME_Coupling_coefficient_pred'])
pop_data = pd.concat([train_data_filter.iloc[:,1:-2],y_pred], axis = 1)

In [ ]:
pop_data

### 5.5 Performing inverse design with GAN-ANN 

Steps for first generation: 5.5.1, 5.5.2, 5.5.3, 5.5.4/5.5.5 <br>
Steps for subsequent generations: 5.5.3, 5.5.4/5.5.5

#### 5.5.1. Generating P samples from GAN

In [ ]:
latent_var = np.random.normal(size=(p,2))
pop_X = generator.predict(latent_var)
latent_var = pd.DataFrame(latent_var,columns=['var1','var2'])
pop_X_cols = ['Sphere 1x','Sphere 1y','Sphere 1z','Sphere 2x','Sphere 2y','Sphere 2z','Sphere 3x','Sphere 3y','Sphere 3z','Sphere 4x','Sphere 4y','Sphere 4z','Sphere 5x','Sphere 5y','Sphere 5z','Sphere 6x','Sphere 6y','Sphere 6z','Sphere 7x','Sphere 7y','Sphere 7z','Sphere 8x','Sphere 8y','Sphere 8z','Sphere 9x','Sphere 9y','Sphere 9z','Sphere 10x','Sphere 10y','Sphere 10z']
pop_X = pd.DataFrame(pop_X,columns = pop_X_cols)
pop_X

In [ ]:
y_pred = pd.DataFrame(ann.predict(pop_X),columns = ['ME_Coupling_coefficient_pred'])

In [ ]:
ga_dataset = pd.concat([latent_var,y_pred],axis=1)
ga_dataset = ga_dataset.sort_values(by=['ME_Coupling_coefficient_pred'], ascending = False).reset_index(drop=True)
ga_dataset['Mating_no'] = round((ga_dataset['ME_Coupling_coefficient_pred']/ga_dataset['ME_Coupling_coefficient_pred'].sum())*p).astype(int)
ga_dataset

Creating mating pool for first time

In [ ]:
ga_dataset['var1'] = round(ga_dataset['var1'],3)
ga_dataset['var2'] = round(ga_dataset['var2'],3)

In [ ]:
if ga_dataset['Mating_no'].sum() < p:
    s = ga_dataset['Mating_no'].sum()
    ga_dataset['Mating_no'][0] += (p-s)
j = 1
while ga_dataset['Mating_no'].sum() > p:
    if ga_dataset['Mating_no'][p-j] == 0:
        j += 1
    ga_dataset['Mating_no'][p-j] -= 1
mate_pool = ga_dataset[ga_dataset['Mating_no']>0][:]
mate_pool

Calculating average fitness for first time

In [ ]:
avg_fitness_hist = []
avg_fitness_hist.append(ga_dataset['ME_Coupling_coefficient_pred'].mean())
print("History of GA:",avg_fitness_hist)

#### 5.5.2. Decimal to binary conversion and vice versa

In [ ]:
def dec_to_binary(num):
    num = int(num*1000)
    binary = []
    abs_num = abs(num)
    while abs_num > 0:
        binary.insert(0,abs_num%2)
        abs_num //= 2
    if num < 0:
        binary.insert(0,1)
    else:
        binary.insert(0,0)
    return binary

def binary_to_dec(binary):
    length = len(binary)
    num = 0
    for i in range(1,length):
        num += (2**(length-i-1))*binary[i]
    num /= 1000
    if binary[0] == 1:
        num = -num
    return num

#### 5.5.3. Genetic Algorithm(to be repeated until convergence) - uses uniform crossover and no mutation

In [ ]:
def GA(mate_pool):
    bin_rep = []
    mate_count = np.shape(mate_pool)[0]
    for i in range(0,mate_count):
        row = []
        row.append(dec_to_binary(mate_pool['var1'][i]))
        row.append(dec_to_binary(mate_pool['var2'][i]))
        for j in range(0,mate_pool['Mating_no'][i]):
            bin_rep.append(row)
    print("Binary_rep before mating")
    for i in range(0,p):
        print(bin_rep[i][0],bin_rep[i][1])
    mate_list = np.arange(0,p)
    random.shuffle(mate_list)
    for i in range(0,p,2):
        partner = i+1
        for j in range(0,2):
            len1 = len(bin_rep[mate_list[i]][j])
            len2 = len(bin_rep[mate_list[partner]][j])
            if len1 > len2:
                for k in range(0,len1-len2):
                    bin_rep[mate_list[partner]][j].insert(0,0)
            elif len1 < len2:
                for k in range(0,len2-len1):
                    bin_rep[mate_list[i]][j].insert(0,0)
            cross_over = [random.choice([0,1]) for k in range(0,max(len1,len2))]
            for k in range(0,max(len1,len2)):
                if cross_over[k] == 1:
                    temp = bin_rep[mate_list[i]][j][k]
                    bin_rep[mate_list[i]][j][k] = bin_rep[mate_list[partner]][j][k]
                    bin_rep[mate_list[partner]][j][k] = temp
    print("Binary_rep after mating")
    for i in range(0,p):
        print(bin_rep[i][0],bin_rep[i][1])
    new_pop = []
    for i in range(0,p):
        row = []
        for j in range(0,2):
            row.append(binary_to_dec(bin_rep[i][j]))
        new_pop.append(row)
    new_pop = np.array(new_pop)
    new_pop = pd.DataFrame(new_pop,columns=['var1','var2'])
    print(new_pop)
    return new_pop

In [ ]:
new_pop_latent = GA(mate_pool)
new_pop_X = pd.DataFrame(generator.predict(new_pop_latent),columns = pop_X_cols)
y_pred = pd.DataFrame(ann.predict(new_pop_X),columns = ['ME_Coupling_coefficient_pred'])
new_pop_data = pd.concat([new_pop_latent,y_pred],axis = 1)
new_pop_data = new_pop_data.sort_values(by=['ME_Coupling_coefficient_pred'], ascending = False).reset_index(drop=True)
new_pop_data

In [ ]:
avg_fitness_hist.append(new_pop_data['ME_Coupling_coefficient_pred'].mean())
print("History of GA:",avg_fitness_hist)

#### 5.5.4 If ME_coupling coefficient shows signs of convergence

In [ ]:
new_pop_data = pd.concat([new_pop_X,y_pred],axis = 1).sort_values(by=['ME_Coupling_coefficient_pred'], ascending = False)
new_pop_data.to_csv(r"Population_data 3\Pop_data2.csv", index = False)
parent_X = new_pop_X

In [ ]:
new_pop_X

#### 5.5.5 If ME coupling coefficient does not show signs of convergence

In [ ]:
ga_dataset = new_pop_data
ga_dataset['Mating_no'] = round((ga_dataset['ME_Coupling_coefficient_pred']/ga_dataset['ME_Coupling_coefficient_pred'].sum())*p).astype(int)
ga_dataset['var1'] = round(ga_dataset['var1'],3)
ga_dataset['var2'] = round(ga_dataset['var2'],3)
if ga_dataset['Mating_no'].sum() < p:
    s = ga_dataset['Mating_no'].sum()
    ga_dataset['Mating_no'][0] += (p-s)
j = 1
while ga_dataset['Mating_no'].sum() > p:
    if ga_dataset['Mating_no'][p-j] == 0:
        j += 1
    ga_dataset['Mating_no'][p-j] -= 1
mate_pool = ga_dataset[ga_dataset['Mating_no']>0][:]
mate_pool

### 5.6. Selecting the top designs 

In [ ]:
pop_data = pop_data.sort_values(by=['ME_Coupling_coefficient_pred'], ascending = False)
parent_data = pop_data.head(M)
parent_data = parent_data.reset_index(drop = True)
parent_data.to_csv(r"Parent_data 3\parent_data0.csv", index = False)
parent_X = parent_data.iloc[:,:-1]
parent_y = parent_data.iloc[:,-1]

### 5.7. Performing expansion of  dataset

In [ ]:
extend_X = parent_X
for i in range(0,44):
    mutated_set = parent_X  + np.random.normal(size = 30)
    extend_X = pd.concat([extend_X ,mutated_set])
extend_X = extend_X.reset_index(drop=True)

In [ ]:
extend_X

In [ ]:
for col in extend_X.columns:
    extend_X.loc[extend_X[col] > 30, col] = 30
    extend_X.loc[extend_X[col] < -30, col] = -30

In [ ]:
extend_X

### 5.8. Predicting ME coupling coefficient in expanded population

In [ ]:
y_pred_extend = pd.DataFrame(ann.predict(extend_X),columns = ['ME_Coupling_coefficient_pred'])
extend_pop = pd.concat([extend_X,y_pred_extend],axis=1)

In [ ]:
extend_pop

### 5.9. Selecting top designs from extended population

In [ ]:
extend_pop = extend_pop.sort_values(by=['ME_Coupling_coefficient_pred'], ascending=False)
parent_data = extend_pop.head(M).reset_index(drop = True)
parent_data.to_csv(r"Parent_data 3\parent_data3.csv", index = False)
parent_X = parent_data.iloc[:,:-1]

In [ ]:
parent_data = pd.read_csv(r"Parent_data 2\parent_data4.csv")
parent_X = parent_data.iloc[:,:-1]

In [ ]:
parent_X

### 5.10. Training GAN with the parent data

In [ ]:
train_hist = compositegan.fit(parent_X,epochs=2000, batch_size = 30)

#### 5.10.1. Performance Review

In [ ]:
plt.suptitle('Generator - Discriminator Loss')
plt.plot(train_hist.history['d_loss'], label='Discriminator_loss')
plt.plot(train_hist.history['g_loss'], label='Generator_loss')
plt.legend()
plt.show()

### 5.11. Sampling data for ANN

In [ ]:
train_data_ANN = parent_data.sample(N).reset_index(drop=True)
X = train_data_ANN.iloc[:,:-1]
X.to_csv(r"Training data 3\ME_Composite_Design_ANN_round3.csv",index=False)

### 5.12. Importing complete data for ANN

In [ ]:
train_data_ANN = pd.read_csv(r"Training data 2\ME_Composite_Design_ANN_combined.csv")
train_data_ANN = train_data_ANN.sample(frac=1)
X = train_data_ANN.iloc[:,1:-1]
y = train_data_ANN.iloc[:,-1]

### 5.13. Training a new ANN with training data from all design rounds except last round

#### 5.13.1. Importing combined training dataset from all design rounds

In [ ]:
train_data_ANN = pd.read_csv(r"Training data 2\ME_Composite_Design_ANN_combined.csv")
train_data_ANN = train_data_ANN.sample(frac=1)
X = train_data_ANN.iloc[:,1:-1]
y = train_data_ANN.iloc[:,-1]

#### 5.13.2. Training a new ANN model

In [ ]:
ann_hist = ann.fit(X,y,epochs=200,batch_size = 10)

#### 5.13.2.1. ANN loss plot

In [ ]:
plt.suptitle('Regression loss')
plt.plot(ann_hist.history['root_mean_squared_error'], label='Regression_loss')
plt.legend()
plt.show()

#### 5.13.2.2. Saving the new ANN model

In [ ]:
ann.save("ann_combined.h5")

### 5.14.  Prediction of ME coupling coefficients of parent and ANN training data with retrained ANN model

#### 5.14.1. Loading the last population data set

In [ ]:
pop_data = pd.read_csv(r"Population_data 2\pop_data3.csv")
pop_X = pop_data.iloc[:,:-1]

In [ ]:
pop_X

#### 5.14.2. Predicting coupling coefficient using retrained ANN

In [ ]:
y_pred_pop = pd.DataFrame(ann.predict(pop_X),columns = ['ME_Coupling_coefficient_pred_corrected'])
pop_data = pd.concat([pop_data,y_pred_pop],axis=1)
pop_data

In [ ]:
pop_data.to_csv(r"Population_data 2\pop_data1_corrected.csv", index = False)

# 6.Saving models in design rounds

In [ ]:
generator.save('generator.h5')
discriminator.save('discriminator.h5')
ann.save('ann.h5')